In [17]:
import pandas as pd
import numpy as np
import pickle
import joblib
import os
import warnings
from typing import Dict, List, Optional, Union, Tuple
import logging
from datetime import datetime

# Core ML libraries
from sentence_transformers import SentenceTransformer
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multioutput import MultiOutputClassifier


In [21]:
# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TourismRecommendationSystem:
    """
    SBERT + XGBoost 기반 강원도 관광지 추천 시스템

    TF-IDF + LogisticRegression → SBERT + XGBoost 업그레이드
    기존 recommend_places() 함수 시그니처 완전 호환
    """

    def __init__(self, data_path: str, model_name: str = 'jhgan/ko-sroberta-multitask'):
        """
        추천 시스템 초기화

        Args:
            data_path: CSV 파일 경로 (gangwon_places_100_enriched.csv)
            model_name: SBERT 모델명 (한국어 KoBERT)
        """
        self.data_path = data_path
        self.model_name = model_name

        # 데이터 및 모델 초기화
        self.df = None
        self.sbert_model = None
        self.place_embeddings = None

        # XGBoost 모델들
        self.season_model = None
        self.nature_model = None
        self.vibe_model = None
        self.target_model = None

        # 인코더들
        self.season_encoder = LabelEncoder()
        self.nature_encoder = MultiLabelBinarizer()
        self.vibe_encoder = MultiLabelBinarizer()
        self.target_encoder = MultiLabelBinarizer()

        # 가중치 설정 (코사인 유사도 0.6 + 태그 매칭 0.4)
        self.similarity_weight = 0.6
        self.tag_weight = 0.4

        # 시스템 초기화
        self._initialize_system()

    def _initialize_system(self):
        """시스템 전체 초기화"""
        logger.info("🚀 관광지 추천 시스템 초기화 시작...")

        # 1. 데이터 로드 및 전처리
        self._load_and_preprocess_data()

        # 2. SBERT 모델 로드
        self._load_sbert_model()

        # 3. 관광지 임베딩 생성
        self._generate_place_embeddings()

        # 4. XGBoost 모델 훈련
        self._train_xgboost_models()

        logger.info("✅ 시스템 초기화 완료!")

    def _load_and_preprocess_data(self):
        """CSV 데이터 로드 및 전처리"""
        logger.info("📁 데이터 로드 중...")

        try:
            self.df = pd.read_csv(self.data_path)
            logger.info(f"데이터 로드 완료: {len(self.df)}개 관광지")
        except FileNotFoundError:
            raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다: {self.data_path}")

        # 데이터 전처리
        self._preprocess_data()

    def _preprocess_data(self):
        """데이터 전처리: 문자열 → 리스트 변환"""
        logger.info("🔧 데이터 전처리 중...")

        # 필수 컬럼 확인
        required_cols = ['name', 'season', 'nature', 'vibe', 'target', 'short_description']
        missing_cols = [col for col in required_cols if col not in self.df.columns]
        if missing_cols:
            raise ValueError(f"필수 컬럼이 누락되었습니다: {missing_cols}")

        # nature, vibe, target을 쉼표 구분 문자열에서 리스트로 변환
        for col in ['nature', 'vibe', 'target']:
            self.df[col] = self.df[col].apply(self._string_to_list)

        # 결측치 처리
        self.df['short_description'] = self.df['short_description'].fillna('')
        self.df['season'] = self.df['season'].fillna('사계절')

        logger.info("✅ 데이터 전처리 완료")

    def _string_to_list(self, value) -> List[str]:
        """문자열을 리스트로 변환 (쉼표 구분)"""
        if pd.isna(value) or value == '':
            return []
        if isinstance(value, list):
            return value
        return [item.strip() for item in str(value).split(',') if item.strip()]

    def _load_sbert_model(self):
        """SBERT 모델 로드 (안전한 폴백 처리)"""
        logger.info(f"🤖 SBERT 모델 로드 중: {self.model_name}")

        # 안정적인 한국어 모델 우선순위
        model_candidates = [
            self.model_name,
            'jhgan/ko-sroberta-multitask',
            'jhgan/ko-sbert-multitask',
            'paraphrase-multilingual-MiniLM-L12-v2'
        ]

        for model_name in model_candidates:
            try:
                logger.info(f"모델 시도 중: {model_name}")
                self.sbert_model = SentenceTransformer(model_name)
                self.model_name = model_name  # 성공한 모델명으로 업데이트
                logger.info(f"✅ SBERT 모델 로드 완료: {model_name}")
                return
            except Exception as e:
                logger.warning(f"모델 {model_name} 로드 실패: {e}")
                continue

        raise RuntimeError("모든 SBERT 모델 로드에 실패했습니다.")

    def _generate_place_embeddings(self):
        """관광지 설명에 대한 SBERT 임베딩 생성 (안전한 처리)"""
        logger.info("🔮 관광지 임베딩 생성 중...")

        descriptions = self.df['short_description'].fillna('').astype(str).tolist()

        # 빈 설명 처리
        descriptions = [desc if desc.strip() else "관광지" for desc in descriptions]

        try:
            # 배치 처리로 임베딩 생성
            batch_size = 16  # 작은 배치 크기로 안정성 향상
            embeddings = []

            for i in range(0, len(descriptions), batch_size):
                batch = descriptions[i:i + batch_size]
                logger.info(f"임베딩 배치 처리 중: {i+1}-{min(i+batch_size, len(descriptions))}/{len(descriptions)}")

                # 각 텍스트의 길이 제한 (토크나이저 오류 방지)
                batch = [text[:500] if len(text) > 500 else text for text in batch]

                batch_embeddings = self.sbert_model.encode(
                    batch,
                    normalize_embeddings=True,
                    show_progress_bar=False,
                    convert_to_tensor=False,  # NumPy 배열로 반환
                    device='cpu'  # CPU 사용으로 안정성 향상
                )
                embeddings.extend(batch_embeddings)

            self.place_embeddings = np.array(embeddings)
            logger.info(f"✅ 임베딩 생성 완료: {self.place_embeddings.shape}")

        except Exception as e:
            logger.error(f"임베딩 생성 실패: {e}")
            # 폴백: 랜덤 임베딩 생성
            logger.warning("랜덤 임베딩으로 대체합니다.")
            embedding_dim = 384  # 기본 차원
            self.place_embeddings = np.random.rand(len(descriptions), embedding_dim)
            logger.info(f"랜덤 임베딩 생성: {self.place_embeddings.shape}")

    def _prepare_training_data(self) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
        """XGBoost 훈련용 데이터 준비"""
        logger.info("📊 훈련 데이터 준비 중...")

        # 특성: SBERT 임베딩
        X = self.place_embeddings

        # 라벨 준비
        labels = {}

        # Season (단일 라벨)
        labels['season'] = self.season_encoder.fit_transform(self.df['season'])

        # Nature, Vibe, Target (다중 라벨)
        labels['nature'] = self.nature_encoder.fit_transform(self.df['nature'])
        labels['vibe'] = self.vibe_encoder.fit_transform(self.df['vibe'])
        labels['target'] = self.target_encoder.fit_transform(self.df['target'])

        logger.info("✅ 훈련 데이터 준비 완료")
        return X, labels

    def _train_xgboost_models(self):
        """XGBoost 모델들 훈련 (안전한 처리)"""
        logger.info("🏋️ XGBoost 모델 훈련 시작...")

        try:
            X, y = self._prepare_training_data()

            # 데이터 검증
            if X.shape[0] < 2:
                logger.warning("데이터가 부족합니다. 최소 2개 이상의 샘플이 필요합니다.")
                return

            # Season 모델 (multi:softmax)
            logger.info("계절 분류기 훈련 중...")
            n_classes = len(np.unique(y['season']))

            if n_classes > 1:
                self.season_model = XGBClassifier(
                    objective='multi:softprob' if n_classes > 2 else 'binary:logistic',
                    n_estimators=min(150, len(X) * 2),  # 데이터 크기에 맞게 조정
                    max_depth=min(5, max(2, len(X) // 10)),
                    learning_rate=0.1,
                    random_state=42,
                    tree_method='hist',
                    verbosity=0  # 로그 출력 최소화
                )
                self.season_model.fit(X, y['season'])
            else:
                logger.warning("계절 데이터에 클래스가 1개뿐입니다. 더미 모델을 사용합니다.")
                from sklearn.dummy import DummyClassifier
                self.season_model = DummyClassifier(strategy='most_frequent')
                self.season_model.fit(X, y['season'])

            # 다중 라벨 모델들 (binary:logistic)
            for label_name in ['nature', 'vibe', 'target']:
                logger.info(f"{label_name} 분류기 훈련 중...")

                # 라벨이 모두 0인지 확인
                if y[label_name].sum() == 0:
                    logger.warning(f"{label_name} 라벨이 모두 비어있습니다. 더미 모델을 사용합니다.")
                    from sklearn.dummy import DummyClassifier
                    dummy_model = DummyClassifier(strategy='constant', constant=0)
                    # MultiOutputClassifier 형태로 래핑
                    model = MultiOutputClassifier(dummy_model)
                    model.fit(X, y[label_name])
                else:
                    base_model = XGBClassifier(
                        objective='binary:logistic',
                        n_estimators=min(150, len(X) * 2),
                        max_depth=min(5, max(2, len(X) // 10)),
                        learning_rate=0.1,
                        random_state=42,
                        tree_method='hist',
                        verbosity=0
                    )

                    # MultiOutputClassifier로 다중 라벨 처리
                    model = MultiOutputClassifier(base_model, n_jobs=1)  # n_jobs=1로 안정성 향상
                    model.fit(X, y[label_name])

                setattr(self, f"{label_name}_model", model)

            logger.info("✅ 모든 XGBoost 모델 훈련 완료!")

        except Exception as e:
            logger.error(f"XGBoost 모델 훈련 실패: {e}")
            # 폴백: 더미 모델들 생성
            logger.warning("더미 모델들로 대체합니다.")
            self._create_dummy_models()

    def _create_dummy_models(self):
        """더미 모델들 생성 (폴백용)"""
        from sklearn.dummy import DummyClassifier

        X, y = self._prepare_training_data()

        # Season 더미 모델
        self.season_model = DummyClassifier(strategy='most_frequent')
        self.season_model.fit(X, y['season'])

        # 다중 라벨 더미 모델들
        for label_name in ['nature', 'vibe', 'target']:
            dummy_model = DummyClassifier(strategy='constant', constant=0)
            model = MultiOutputClassifier(dummy_model)
            model.fit(X, y[label_name])
            setattr(self, f"{label_name}_model", model)

    def _encode_user_input(self, user_input: Dict[str, Union[str, List[str]]]) -> np.ndarray:
        """사용자 입력을 SBERT 임베딩으로 변환 (안전한 처리)"""
        try:
            # 사용자 입력을 하나의 텍스트로 결합
            combined_text = f"{user_input.get('season', '')} "

            for key in ['nature', 'vibe', 'target']:
                if key in user_input:
                    values = user_input[key]
                    if isinstance(values, list):
                        combined_text += ' '.join(str(v) for v in values) + ' '
                    else:
                        combined_text += str(values) + ' '

            # 텍스트 정리
            combined_text = combined_text.strip()
            if not combined_text:
                combined_text = "관광지"

            # 길이 제한
            if len(combined_text) > 500:
                combined_text = combined_text[:500]

            # SBERT 임베딩 생성
            user_embedding = self.sbert_model.encode(
                [combined_text],
                normalize_embeddings=True,
                convert_to_tensor=False,
                device='cpu'
            )
            return user_embedding[0]

        except Exception as e:
            logger.error(f"사용자 입력 인코딩 실패: {e}")
            # 폴백: 랜덤 임베딩
            embedding_dim = self.place_embeddings.shape[1] if self.place_embeddings is not None else 384
            return np.random.rand(embedding_dim)

    def _predict_categories(self, user_embedding: np.ndarray) -> Dict[str, Union[str, List[str]]]:
        """사용자 임베딩으로부터 카테고리 예측 (안전한 처리)"""
        user_embedding_2d = user_embedding.reshape(1, -1)
        predictions = {}

        try:
            # Season 예측
            if self.season_model is not None:
                season_pred = self.season_model.predict(user_embedding_2d)[0]
                predictions['season'] = self.season_encoder.inverse_transform([season_pred])[0]
            else:
                predictions['season'] = '사계절'

            # 다중 라벨 예측
            for label_name in ['nature', 'vibe', 'target']:
                model = getattr(self, f"{label_name}_model", None)
                encoder = getattr(self, f"{label_name}_encoder", None)

                if model is not None and encoder is not None:
                    try:
                        pred = model.predict(user_embedding_2d)
                        predicted_labels = encoder.inverse_transform(pred)[0]
                        predictions[label_name] = list(predicted_labels)
                    except Exception as e:
                        logger.warning(f"{label_name} 예측 실패: {e}")
                        predictions[label_name] = []
                else:
                    predictions[label_name] = []

        except Exception as e:
            logger.error(f"카테고리 예측 실패: {e}")
            # 기본값 반환
            predictions = {
                'season': '사계절',
                'nature': [],
                'vibe': [],
                'target': []
            }

        return predictions

    def _calculate_similarity_scores(self, user_embedding: np.ndarray) -> np.ndarray:
        """사용자 임베딩과 모든 관광지 간 코사인 유사도 계산 (안전한 처리)"""
        try:
            user_embedding_2d = user_embedding.reshape(1, -1)

            # 임베딩 차원 확인 및 조정
            if user_embedding_2d.shape[1] != self.place_embeddings.shape[1]:
                logger.warning(f"임베딩 차원 불일치: {user_embedding_2d.shape[1]} vs {self.place_embeddings.shape[1]}")
                # 차원 맞추기
                min_dim = min(user_embedding_2d.shape[1], self.place_embeddings.shape[1])
                user_embedding_2d = user_embedding_2d[:, :min_dim]
                place_embeddings_adjusted = self.place_embeddings[:, :min_dim]
            else:
                place_embeddings_adjusted = self.place_embeddings

            similarities = cosine_similarity(user_embedding_2d, place_embeddings_adjusted)[0]

            # NaN 값 처리
            similarities = np.nan_to_num(similarities, nan=0.0)

            return similarities

        except Exception as e:
            logger.error(f"유사도 계산 실패: {e}")
            # 폴백: 랜덤 유사도
            return np.random.rand(len(self.df)) * 0.1  # 낮은 점수

    def _calculate_tag_matching_scores(self, user_input: Dict, predicted_categories: Dict) -> np.ndarray:
        """태그 매칭 점수 계산 (안전한 처리)"""
        scores = np.zeros(len(self.df))

        try:
            for idx, row in self.df.iterrows():
                score = 0

                # Season 매칭
                user_season = user_input.get('season', '')
                place_season = row.get('season', '')
                if user_season and place_season and user_season == place_season:
                    score += 1

                # 다중 라벨 매칭 (Jaccard 유사도)
                for category in ['nature', 'vibe', 'target']:
                    if category in user_input:
                        user_tags = user_input[category]
                        if isinstance(user_tags, str):
                            user_tags = [user_tags]
                        elif not isinstance(user_tags, list):
                            user_tags = []

                        user_tags = set(str(tag).strip() for tag in user_tags if tag)

                        place_tags = row.get(category, [])
                        if isinstance(place_tags, str):
                            place_tags = [place_tags]
                        elif not isinstance(place_tags, list):
                            place_tags = []

                        place_tags = set(str(tag).strip() for tag in place_tags if tag)

                        if user_tags and place_tags:
                            intersection = len(user_tags.intersection(place_tags))
                            union = len(user_tags.union(place_tags))
                            jaccard = intersection / union if union > 0 else 0
                            score += jaccard

                scores[idx] = score

            # 정규화 (0-1 범위)
            if scores.max() > 0:
                scores = scores / scores.max()

        except Exception as e:
            logger.error(f"태그 매칭 점수 계산 실패: {e}")
            # 폴백: 균등한 점수
            scores = np.ones(len(self.df)) * 0.5

        return scores

    def recommend_places(self, user_input: Dict[str, Union[str, List[str]]],
                        tfidf_vectorizer=None, season_model=None, nature_model=None,
                        vibe_model=None, target_model=None, df=None, top_n: int = 3) -> pd.DataFrame:
        """
        기존 시그니처와 완전 호환되는 추천 함수

        Args:
            user_input: 사용자 선호도
                - season: str (예: "여름")
                - nature: List[str] (예: ["바다", "자연"])
                - vibe: List[str] (예: ["감성", "산책"])
                - target: List[str] (예: ["연인"])
            tfidf_vectorizer: 호환성용 (사용하지 않음)
            season_model: 호환성용 (사용하지 않음)
            nature_model: 호환성용 (사용하지 않음)
            vibe_model: 호환성용 (사용하지 않음)
            target_model: 호환성용 (사용하지 않음)
            df: 호환성용 (사용하지 않음)
            top_n: 추천할 관광지 수

        Returns:
            추천 관광지 DataFrame (name, city, description 컬럼 포함)
        """
        logger.info(f"🎯 {top_n}개 관광지 추천 생성 중...")

        try:
            # 1. 사용자 입력을 SBERT 임베딩으로 변환
            user_embedding = self._encode_user_input(user_input)

            # 2. 카테고리 예측 (참고용)
            predicted_categories = self._predict_categories(user_embedding)
            logger.info(f"예측된 카테고리: {predicted_categories}")

            # 3. 코사인 유사도 계산
            similarity_scores = self._calculate_similarity_scores(user_embedding)

            # 4. 태그 매칭 점수 계산
            tag_scores = self._calculate_tag_matching_scores(user_input, predicted_categories)

            # 5. 최종 점수 계산 (가중 평균)
            final_scores = (
                self.similarity_weight * similarity_scores +
                self.tag_weight * tag_scores
            )

            # 6. 상위 N개 추천
            top_indices = np.argsort(final_scores)[::-1][:top_n]

            # 7. 결과 DataFrame 생성 (기존 포맷 호환)
            recommendations = self.df.iloc[top_indices].copy()

            # 추천 점수 추가
            recommendations['similarity_score'] = similarity_scores[top_indices]
            recommendations['tag_score'] = tag_scores[top_indices]
            recommendations['final_score'] = final_scores[top_indices]

            # 기존 호환 형식으로 반환 (name, city, description)
            # city 컬럼이 없으면 빈 값으로 추가
            if 'city' not in recommendations.columns:
                recommendations['city'] = ''

            # description이 없으면 short_description 사용
            if 'description' not in recommendations.columns:
                recommendations['description'] = recommendations['short_description']

            result = recommendations[['name', 'city', 'description']].reset_index(drop=True)

            logger.info("✅ 추천 생성 완료!")
            return result

        except Exception as e:
            logger.error(f"추천 생성 실패: {e}")
            # 폴백: 첫 번째 N개 관광지 반환
            fallback_df = self.df.head(top_n).copy()
            if 'city' not in fallback_df.columns:
                fallback_df['city'] = ''
            if 'description' not in fallback_df.columns:
                fallback_df['description'] = fallback_df['short_description']

            return fallback_df[['name', 'city', 'description']].reset_index(drop=True)

    def get_detailed_recommendations(self, user_input: Dict[str, Union[str, List[str]]],
                                   top_n: int = 5) -> pd.DataFrame:
        """상세 정보가 포함된 추천 결과 반환"""
        logger.info(f"🔍 상세 추천 정보 생성 중...")

        user_embedding = self._encode_user_input(user_input)
        predicted_categories = self._predict_categories(user_embedding)
        similarity_scores = self._calculate_similarity_scores(user_embedding)
        tag_scores = self._calculate_tag_matching_scores(user_input, predicted_categories)
        final_scores = (
            self.similarity_weight * similarity_scores +
            self.tag_weight * tag_scores
        )

        top_indices = np.argsort(final_scores)[::-1][:top_n]
        recommendations = self.df.iloc[top_indices].copy()

        # 점수 정보 추가
        recommendations['similarity_score'] = similarity_scores[top_indices]
        recommendations['tag_score'] = tag_scores[top_indices]
        recommendations['final_score'] = final_scores[top_indices]
        recommendations['predicted_season'] = predicted_categories['season']
        recommendations['predicted_nature'] = str(predicted_categories['nature'])
        recommendations['predicted_vibe'] = str(predicted_categories['vibe'])
        recommendations['predicted_target'] = str(predicted_categories['target'])

        return recommendations.reset_index(drop=True)

    def save_models(self, model_dir: str):
        """모델들을 지정된 디렉토리에 저장"""
        logger.info(f"💾 모델 저장 중: {model_dir}")

        os.makedirs(model_dir, exist_ok=True)

        # SBERT 임베딩 저장
        np.save(os.path.join(model_dir, 'place_embeddings.npy'), self.place_embeddings)

        # XGBoost 모델들 저장
        models_to_save = {
            'season_model': self.season_model,
            'nature_model': self.nature_model,
            'vibe_model': self.vibe_model,
            'target_model': self.target_model
        }

        for name, model in models_to_save.items():
            if model is not None:
                joblib.dump(model, os.path.join(model_dir, f'{name}.joblib'))

        # 인코더들 저장
        encoders_to_save = {
            'season_encoder': self.season_encoder,
            'nature_encoder': self.nature_encoder,
            'vibe_encoder': self.vibe_encoder,
            'target_encoder': self.target_encoder
        }

        for name, encoder in encoders_to_save.items():
            joblib.dump(encoder, os.path.join(model_dir, f'{name}.joblib'))

        # 데이터프레임과 설정 저장
        self.df.to_csv(os.path.join(model_dir, 'processed_data.csv'), index=False)

        config = {
            'model_name': self.model_name,
            'similarity_weight': self.similarity_weight,
            'tag_weight': self.tag_weight,
            'data_path': self.data_path
        }

        with open(os.path.join(model_dir, 'config.pickle'), 'wb') as f:
            pickle.dump(config, f)

        logger.info("✅ 모델 저장 완료!")

    def load_models(self, model_dir: str):
        """저장된 모델들을 로드"""
        logger.info(f"📂 모델 로드 중: {model_dir}")

        # 설정 로드
        with open(os.path.join(model_dir, 'config.pickle'), 'rb') as f:
            config = pickle.load(f)

        self.model_name = config['model_name']
        self.similarity_weight = config['similarity_weight']
        self.tag_weight = config['tag_weight']

        # 데이터 로드
        self.df = pd.read_csv(os.path.join(model_dir, 'processed_data.csv'))

        # nature, vibe, target 컬럼을 다시 리스트로 변환
        for col in ['nature', 'vibe', 'target']:
            self.df[col] = self.df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

        # SBERT 모델 로드
        self.sbert_model = SentenceTransformer(self.model_name)

        # 임베딩 로드
        self.place_embeddings = np.load(os.path.join(model_dir, 'place_embeddings.npy'))

        # XGBoost 모델들 로드
        model_names = ['season_model', 'nature_model', 'vibe_model', 'target_model']
        for name in model_names:
            model_path = os.path.join(model_dir, f'{name}.joblib')
            if os.path.exists(model_path):
                setattr(self, name, joblib.load(model_path))

        # 인코더들 로드
        encoder_names = ['season_encoder', 'nature_encoder', 'vibe_encoder', 'target_encoder']
        for name in encoder_names:
            encoder_path = os.path.join(model_dir, f'{name}.joblib')
            if os.path.exists(encoder_path):
                setattr(self, name, joblib.load(encoder_path))

        logger.info("✅ 모델 로드 완료!")

    def evaluate_models(self) -> Dict[str, float]:
        """모델 성능 평가"""
        logger.info("📊 모델 성능 평가 중...")

        X = self.place_embeddings
        results = {}

        # Season 모델 평가
        y_season = self.season_encoder.transform(self.df['season'])
        season_pred = self.season_model.predict(X)
        results['season_accuracy'] = (y_season == season_pred).mean()

        # 다중 라벨 모델들 평가
        for label_name in ['nature', 'vibe', 'target']:
            encoder = getattr(self, f"{label_name}_encoder")
            model = getattr(self, f"{label_name}_model")

            y_true = encoder.transform(self.df[label_name])
            y_pred = model.predict(X)

            f1 = f1_score(y_true, y_pred, average='micro')
            results[f'{label_name}_f1_score'] = f1

        logger.info("평가 결과:")
        for metric, score in results.items():
            logger.info(f"  {metric}: {score:.4f}")

        return results

In [26]:
# 기존 함수 시그니처와 완전 호환되는 래퍼 함수
def recommend_places(user_input: Dict[str, Union[str, List[str]]],
                    tfidf_vectorizer=None, season_model=None, nature_model=None,
                    vibe_model=None, target_model=None, df=None, top_n: int = 3,
                    system: TourismRecommendationSystem = None) -> pd.DataFrame:
    """
    기존 코드와 완전 호환되는 추천 함수

    새로운 시스템을 사용하려면 system 파라미터에 TourismRecommendationSystem 인스턴스를 전달
    """
    if system is None:
        raise ValueError("TourismRecommendationSystem 인스턴스를 system 파라미터로 전달해주세요.")

    return system.recommend_places(
        user_input=user_input,
        tfidf_vectorizer=tfidf_vectorizer,
        season_model=season_model,
        nature_model=nature_model,
        vibe_model=vibe_model,
        target_model=target_model,
        df=df,
        top_n=top_n
    )


# 사용 예시 및 테스트 코드
def demo_usage():
    """사용 예시 데모 (안전한 처리)"""
    print("🌟 강원도 관광지 추천 시스템 - SBERT + XGBoost 버전")
    print("=" * 60)

    # Google Drive 마운트 시도
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive 마운트 완료")

        # 사용자 지정 CSV 파일 경로
        data_path = "/content/drive/My Drive/졸업논문/gangwon_places_100.csv"

    except ImportError:
        print("ℹ️ Google Colab 환경이 아닙니다.")
        data_path = "/content/drive/My Drive/졸업논문/gangwon_places_100.csv"

    # 샘플 데이터 생성
    if not os.path.exists(data_path):
        print(f"❌ 파일을 찾을 수 없습니다: {data_path}")
        print("💡 올바른 파일 경로를 확인해주세요.")
        return
    else:
        print(f"✅ 실제 데이터 파일 발견: 100개 관광지")

    try:
        # 1. 시스템 초기화 (실제 100개 관광지 데이터 사용)
        print(f"\n🔧 시스템 초기화 중... (데이터: {data_path})")
        system = TourismRecommendationSystem(data_path)

        # 2. 사용자 입력 예시
        user_input = {
            "season": "겨울",
            "nature": ["바다", "자연"],
            "vibe": ["감성", "산책"],
            "target": ["연인"]
        }

        print(f"\n🔍 사용자 선호도: {user_input}")

        # 3. 기존 함수 시그니처로 추천 실행
        print("\n📋 기존 시그니처 호환 테스트:")
        recommendations = system.recommend_places(
            user_input=user_input,
            tfidf_vectorizer=None,  # 더 이상 사용하지 않음
            season_model=None,      # 더 이상 사용하지 않음
            nature_model=None,      # 더 이상 사용하지 않음
            vibe_model=None,        # 더 이상 사용하지 않음
            target_model=None,      # 더 이상 사용하지 않음
            df=None,               # 더 이상 사용하지 않음
            top_n=5  # 🎯 추천 개수: 5개로 변경
        )

        print("🏆 추천 결과:")
        print(recommendations.to_string(index=False))

        # 4. 상세 추천 결과
        print("\n📊 상세 추천 결과:")
        detailed = system.get_detailed_recommendations(user_input, top_n=5)  # 🎯 상세 추천도 5개로 변경
        print(detailed[['name', 'season', 'final_score', 'similarity_score', 'tag_score']].to_string(index=False))

        # 5. 모델 성능 평가
        print("\n📈 모델 성능:")
        performance = system.evaluate_models()

        # 6. 모델 저장 테스트
        print("\n💾 모델 저장 테스트:")
        try:
            system.save_models("saved_models")
            print("✅ 모델 저장 완료!")

            # 새 인스턴스로 로드 테스트
            print("🔄 모델 로드 테스트...")
            new_system = TourismRecommendationSystem.__new__(TourismRecommendationSystem)
            new_system.load_models("saved_models")
            print("✅ 모델 로드 완료!")

            # 로드된 시스템으로 추천 테스트
            test_rec = new_system.recommend_places(user_input=user_input, top_n=5)  # 🎯 로드 테스트도 5개로 변경
            print("✅ 로드된 모델 추천 테스트 완료!")

        except Exception as e:
            print(f"⚠️ 모델 저장/로드 경고: {e}")

        print("\n🎉 모든 테스트 완료!")

    except Exception as e:
        print(f"❌ 시스템 초기화 실패: {e}")
        print("상세 오류:")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    demo_usage()

🌟 강원도 관광지 추천 시스템 - SBERT + XGBoost 버전
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 마운트 완료
✅ 실제 데이터 파일 발견: 100개 관광지

🔧 시스템 초기화 중... (데이터: /content/drive/My Drive/졸업논문/gangwon_places_100.csv)

🔍 사용자 선호도: {'season': '겨울', 'nature': ['바다', '자연'], 'vibe': ['감성', '산책'], 'target': ['연인']}

📋 기존 시그니처 호환 테스트:
🏆 추천 결과:
        name city                                                    description
        설악산책               설악산책은(는) 겨울에 특히 아름다워 산 경관이 뛰어나며 (감성) 분위기로 연인에게 추천됩니다.
구봉산 전망대 카페거리       구봉산 전망대 카페거리은(는) 겨울에 특히 아름다워 산 경관이 뛰어나며 (감성) 분위기로 연인에게 추천됩니다.
   이승만별장(고성)         이승만별장(고성)은(는) 겨울에 특히 아름다워 호수 경관이 뛰어나며 (감성) 분위기로 연인에게 추천됩니다.
사근진 해중공원 전망대      사근진 해중공원 전망대은(는) 여름에 특히 아름다워 바다 경관이 뛰어나며 (감성) 분위기로 연인에게 추천됩니다.
        순담계곡               순담계곡은(는) 여름에 특히 아름다워 산 경관이 뛰어나며 (감성) 분위기로 연인에게 추천됩니다.

📊 상세 추천 결과:
        name season  final_score  similarity_score  tag_score
        설악산책     겨울     0.853